In [8]:
import pandas as pd

In [9]:
df = pd.read_csv('camelyon17_sweep.csv')
df = df.dropna()
df = df[df['fpr_id']<= 0.10]

In [10]:
top_k = 5  # You can adjust this

topk_configs = (
    df[df['dpddm.data_sample_size'].isin([10, 20, 50])]
    .sort_values('tpr', ascending=False)
    .groupby('dpddm.data_sample_size')
    .head(top_k)
)

# Optional: look at what hyperparameters are most frequent in top-k
topk_summary = topk_configs.groupby('dpddm.data_sample_size').agg(lambda x: x.mode().iloc[0])
topk_summary


,run_id,name,state,metrics,created_at,dataset,dpddm.temp,dataset.frac,model.prior_scale,model.resnet_type,...,_wandb.runtime,Phi-mean,Phi-med,Phi-std,dr_id,dr_ood,dr_train,fpr_id,fpr_train,tpr
dpddm.data_sample_size,,,,,,,,,,,,,,,,,,,,,
10,2x18gg2i,astral-sweep-1982,finished,"{'Phi-mean': 0.3462000079005957, 'Phi-med': 0....",2025-04-15T18:38:47Z,camelyon17,2,1,2,resnet50,...,1565.0,0.34620,0.50,0.079282,"[0.33300000831484794, 0.0872410454175284]","[0.8090000140666962, 0.07628238523983472]","[0.3390000085532665, 0.08472897916893773]",0.07,0.06,0.99
20,4wpiqfkm,comfy-sweep-2448,finished,"{'Phi-mean': 0.07960000141710043, 'Phi-med': 0...",2025-04-16T00:44:27Z,camelyon17,1,1,2,resnet50,...,946.0,0.07960,0.10,0.030230,"[0.07700000140815973, 0.031160873977000767]","[0.2515000031888485, 0.05676926979813739]","[0.08250000160187483, 0.032691743332876166]",0.04,0.08,1.00
50,6coc9vhk,autumn-sweep-827,finished,"{'Phi-mean': 0.04407999897748232, 'Phi-med': 0...",2025-04-16T00:42:20Z,camelyon17,2,1,1,resnet50,...,785.0,0.04408,0.04,0.017870,"[0.046599998883903024, 0.018773384785048343]","[0.1397999967634678, 0.02792776259146313]","[0.04359999895095825, 0.018629008993552205]",0.10,0.06,1.00


In [13]:
# Define "good" runs: low FPR and high TPR
good_runs = df[(df['fpr_id'] <= 0.1) & (df['tpr'] >= 0.95)]

# Focus on these sample sizes
valid_sizes = [10, 20, 50]

# Hyperparameters to analyze
hyperparams = [
    'model.resnet_type',
    'model.resnet_pretrained',
    'model.prior_scale',
    'model.wishart_scale',
    'model.freeze_features',
    'model.reg_weight_factor',
    'dpddm.temp'
]

# For each sample size, print most common hyperparameters among good runs
for size in valid_sizes:
    subset = good_runs[good_runs['dpddm.data_sample_size'] == size]
    print(f"\n=== Most common GOOD config for data_sample_size = {size} (TPR ≥ 0.95, FPR ≤ 0.1) ===")
    
    if subset.empty:
        print("No good runs found.")
        continue
    
    for param in hyperparams:
        if param in subset.columns:
            most_common = subset[param].mode()
            val = most_common.iloc[0] if not most_common.empty else "N/A"
            count = (subset[param] == val).sum()
            print(f"{param:30s}: {val}  (count: {count})")



=== Most common GOOD config for data_sample_size = 10 (TPR ≥ 0.95, FPR ≤ 0.1) ===
model.resnet_type             : resnet50  (count: 20)
model.resnet_pretrained       : True  (count: 32)
model.prior_scale             : 2  (count: 16)
model.wishart_scale           : 5  (count: 12)
model.freeze_features         : True  (count: 19)
model.reg_weight_factor       : 10  (count: 21)
dpddm.temp                    : 1  (count: 18)

=== Most common GOOD config for data_sample_size = 20 (TPR ≥ 0.95, FPR ≤ 0.1) ===
model.resnet_type             : resnet50  (count: 47)
model.resnet_pretrained       : True  (count: 81)
model.prior_scale             : 1  (count: 30)
model.wishart_scale           : 5  (count: 32)
model.freeze_features         : False  (count: 58)
model.reg_weight_factor       : 10  (count: 35)
dpddm.temp                    : 2  (count: 31)

=== Most common GOOD config for data_sample_size = 50 (TPR ≥ 0.95, FPR ≤ 0.1) ===
model.resnet_type             : resnet34  (count: 77)
model.resn